# KMeans Clustering - Experiment

This is a component that trains a KMeans model using [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html). 
<br>
Scikit-learn is an open source machine learning library that supports supervised and unsupervised learning. It also provides various tools for model fitting, data preprocessing, model selection and evaluation, and many other utilities.

This notebook shows:
- how to use SDK to load the dataset and save a model.
- how to receive parameters from the platform.

## Declare parameters
Components may declare (and use) these default parameters:
- dataset

Use these parameters to load/save datasets, models, metrics, and figures with the help of [PlatIAgro SDK](https://platiagro.github.io/sdk/).

You may also declare custom parameters to set when running an experiment.

In [ ]:
dataset = "iris" #@param {type:"string"}
n_clusters = 3 #@param {type:"integer"}
n_init = 10 #@param {type:"integer"}
max_iter = 300 #@param {type:"integer"}
algorithm = "auto" #@param {type:"string"}

## Load dataset

Import and put the whole dataset in a pandas.DataFrame.

In [ ]:
from platiagro import load_dataset

df = load_dataset(name=dataset)

## Load metadata about the dataset
For example, below we get the feature type for each column in the dataset. (eg. categorical, numerical, or datetime)

In [ ]:
import numpy as np
from platiagro import stat_dataset

metadata = stat_dataset(name=dataset)
featuretypes = metadata["featuretypes"]
featuretypes = np.array(featuretypes)
columns = df.columns.to_numpy()

## Features configuration

In [ ]:
from platiagro.featuretypes import NUMERICAL

# Selects the indexes of numerical and non-numerical features
numerical_indexes = np.where(featuretypes == NUMERICAL)[0]
non_numerical_indexes = np.where(~(featuretypes == NUMERICAL))[0]

# After the step handle_missing_values, 
# numerical features are grouped in the beggining of the array
numerical_indexes_after_handle_missing_values = \
    np.arange(len(numerical_indexes))
non_numerical_indexes_after_handle_missing_values = \
    np.arange(len(numerical_indexes), len(featuretypes))

## Fit a model using sklearn.cluster.KMeans

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from category_encoders.ordinal import OrdinalEncoder

pipeline = Pipeline(steps=[
            ("handle_missing_values",
             ColumnTransformer(
                [("imputer_mean", SimpleImputer(strategy="mean"), numerical_indexes),
                 ("imputer_mode", SimpleImputer(strategy="most_frequent"), non_numerical_indexes)],
                 remainder="drop")),
            ("handle_categorical_features",
             ColumnTransformer(
                 [("feature_encoder", OrdinalEncoder(), non_numerical_indexes_after_handle_missing_values)],
                 remainder="passthrough")),
            ("estimator", KMeans(n_clusters=n_clusters,
                                    n_init=n_init,
                                    max_iter=max_iter,
                                    algorithm=algorithm))
])

X = pipeline.fit_transform(df)

## Measure the performance

In the case of Kmeans we may measure performance by the silhouette score.

In [ ]:
import numpy as np
import pandas as pd
import sklearn.metrics as metrics

labels = pipeline.named_steps.estimator.labels_
score = metrics.silhouette_score(X, labels, metric="euclidean")

In [ ]:
from sklearn.decomposition import PCAfrom sklearn.preprocessing import StandardScaler

X_std = StandardScaler().fit_transform(X.copy())pca = PCA(n_components=2)
reduced = pca.fit_transform(X_std)

X_pca = pd.DataFrame(reduced, columns=["X", "Y"])

X_pca["Cluster"] = labels

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

ax = sns.scatterplot(x="X", y="Y", hue="Cluster", data=X_pca)

ax.set_title("PCA Graph", {"fontweight": "bold"})

In [ ]:
from platiagro import save_figure

save_figure(figure=plt.gcf())

## Save metrics

Record the metrics used to evaluate the model.<br>
It's a good way to document the experiments, and also help to avoid running the same experiment twice. 

In [ ]:
from platiagro import save_metrics

save_metrics(silhouette_score=score)

## Save model

Stores the model artifacts in a object storage.<br>
It will make the model available for future deployments.

In [ ]:
from platiagro import save_model

save_model(pipeline=pipeline, columns=columns)